# Sociolla Scraper

## Dependencies

In [ ]:
import requests
import json
import time
import random

import pandas as pd

from bs4 import BeautifulSoup
from datetime import datetime, timedelta

import sys
from pathlib import Path

# Automatically detect the repo root (parent of notebook folder)
repo_root = Path().resolve().parent  # if notebook is in 'notebooks/' folder
sys.path.append(str(repo_root))

from config.config import get_environment

from config.config import data_import_json, data_export_json, data_import_pandas, data_export_pandas

## ENV

In [ ]:
ENV = get_environment(
    env_path="../environments",
    env_name="env.json"
)

# content_date = datetime.now().date() + timedelta(days=0)
content_date = ENV['CONTENT_DATE']
website = ENV['SOURCE']['NAME']
version = ENV['VERSION']

url_scrape = ENV['SOURCE']['URL']

## Mining

### Get Total Items Count

In [ ]:
headers_count = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.9,id;q=0.8',
    'adjust_device_id': '0de186d9-af6f-44cc-27c5-d0b78ebaec19',
    'origin': 'https://www.sociolla.com',
    'priority': 'u=1, i',
    'referer': 'https://www.sociolla.com/',
    'sec-ch-ua': '"Chromium";v="142", "Google Chrome";v="142", "Not_A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'session_id': 'slwrj5hc-iul1-4nuu-bsty-5vnqflqtmcf5',
    'soc-platform': 'sociolla-web-mobile',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36',
    # 'cookie': 'sso_token=1b6df37b214caba6ac913e935d4e82f644ecfdb4; SOCIOLLA_UTM_DATA={}; SOCIOLLA_SESSION_ID=slwrj5hc-iul1-4nuu-bsty-5vnqflqtmcf5; _gid=GA1.2.1512964316.1764085677; _gcl_au=1.1.2118681921.1764085678; _fbp=fb.1.1764085684064.131487521762752714; _hjSession_1067100=eyJpZCI6IjM0ODRlODg1LTA0NTQtNGUyNi1hMTBlLTA3NmQ4NjhkNmEyZCIsImMiOjE3NjQwODU2ODQxNzMsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjoxLCJzcCI6MX0=; _tt_enable_cookie=1; _ttp=01KAXV6P6S7RFD10XVYF4EB3H4_.tt.1; _hjSessionUser_1067100=eyJpZCI6ImFhODUxY2M1LTUzZjktNTc2Yy05YWNmLWM5MmQyNmJjNjk4ZiIsImNyZWF0ZWQiOjE3NjQwODU2ODQxNzEsImV4aXN0aW5nIjp0cnVlfQ==; ttcsid=1764085684456::wNOTNM-WZilRf-bNQlPi.1.1764085724342.0; ttcsid_C10ST5FMJ1JP7306D860=1764085684455::bHsnIh5Apg41LRMq9u5J.1.1764085724343.0; ttcsid_C7FU0AI0VTGLO2V4FJK0=1764085684457::LhOq5jGx-4rzao-I8Mfe.1.1764085724343.0; _gat=1; _dc_gtm_UA-57294171-1=1; _ga_363PMMDMHM=GS2.1.s1764085678$o1$g1$t1764086157$j57$l0$h0; _ga=GA1.1.744826987.1764085677',
}

url_scrape_count = 'https://catalog-api4.sociolla.com/search/count?limit=16&skip=0&sort=-best-seller&filter=%7B%22categories.id%22:%225d3d50276b24d01599516819%22%7D'

print(f"Get Total Items Count | URL: {url_scrape_count}")

response_count = requests.get(
    url_scrape_count,
    headers=headers_count,
)

print(f"Response Total Items Count | URL: {url_scrape_count} | STATUS CODE: {response_count.status_code}")

try:
    total_items = response_count.json()['data']
    print(f"Total Items: {total_items}")
except Exception as e:
    raise e

### Get Category and Items per Category (Currently Skincare only)

In [ ]:
headers_cat = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.9,id;q=0.8',
    'adjust_device_id': '0de186d9-af6f-44cc-27c5-d0b78ebaec19',
    'origin': 'https://www.sociolla.com',
    'priority': 'u=1, i',
    'referer': 'https://www.sociolla.com/',
    'sec-ch-ua': '"Chromium";v="142", "Google Chrome";v="142", "Not_A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'session_id': 'ytvqy8vv-onte-22sd-1sas-9gt5o4i6gjui',
    'soc-platform': 'sociolla-web-mobile',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36',
    # 'cookie': 'sso_token=1b6df37b214caba6ac913e935d4e82f644ecfdb4; _gid=GA1.2.1512964316.1764085677; _fbp=fb.1.1764085684064.131487521762752714; _tt_enable_cookie=1; _ttp=01KAXV6P6S7RFD10XVYF4EB3H4_.tt.1; _hjSessionUser_1067100=eyJpZCI6ImFhODUxY2M1LTUzZjktNTc2Yy05YWNmLWM5MmQyNmJjNjk4ZiIsImNyZWF0ZWQiOjE3NjQwODU2ODQxNzEsImV4aXN0aW5nIjp0cnVlfQ==; SOCIOLLA_UTM_DATA={}; _hjSession_1067100=eyJpZCI6IjZkYmY5ZGUxLTMwYzctNGVhZS05MTI1LThjMmYyNmZiN2E5MyIsImMiOjE3NjQxMzM0MTQwMDMsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjowLCJzcCI6MH0=; SOCIOLLA_SESSION_ID=ytvqy8vv-onte-22sd-1sas-9gt5o4i6gjui; ttcsid=1764135926535::e_gk012KkFyNpX1LV82t.3.1764137333823.0; ttcsid_C10ST5FMJ1JP7306D860=1764135926536::6QcxrwJgH9eifzn95eX2.3.1764137333823.0; ttcsid_C7FU0AI0VTGLO2V4FJK0=1764135926536::xvA3WuZ503PmR63S0msb.3.1764137333823.0; _gcl_au=1.1.2118681921.1764085678; _gat=1; _dc_gtm_UA-57294171-1=1; _ga_363PMMDMHM=GS2.1.s1764135924$o3$g1$t1764137458$j60$l0$h0; _ga=GA1.1.744826987.1764085677',
}

url_scrape_cat = 'https://catalog-api5.sociolla.com/v3/categories/children?filter=%7B%22name%22:%22Shop+By+Departments%22,%22depth%22:3%7D&fields=name,my_sociolla_sql_id,link_rewrite,parents,slug,banner_for_sociolla,logo&skip=0&limit=30&sort=position'

print(f"Get Item Categories | URL: {url_scrape_cat}")

response_cat = requests.get(
    url_scrape_cat,
    headers=headers_cat,
)

print(f"Response Item Categories | URL: {url_scrape_cat} | STATUS CODE: {response_cat.status_code}")

# Filter only Skincare
try:
    item_cat = [d for d in response_cat.json()['data']['children'] if d['name'] in ['Skincare']][0]
    print(f"Total Categories: {len(item_cat['children'])}")
except Exception as e:
    raise e

# Loop count and id dependencies for categories within Skincare
cat_count_list = list()
for index, cat in enumerate(item_cat['children']):

    url_cat_count = f'https://catalog-api4.sociolla.com/search/count?limit=16&skip=0&sort=-best-seller&filter=%7B%22categories.id%22:%22{cat["_id"]}%22%7D'

    print(f"Get Total Items Count {index+1}/{len(item_cat['children'])} | URL: {url_cat_count}")

    response_cat_count = requests.get(
        url_cat_count,
        headers=headers_count,
    )

    print(f"Response Category Items Count {index+1}/{len(item_cat['children'])} | URL: {url_cat_count} | STATUS CODE: {response_cat_count.status_code}")

    cat_dict = dict()
    try:
        cat_count = response_cat_count.json()['data']
        cat_dict['status_code'] = response_cat_count.status_code
        cat_dict['id'] = cat['_id']
        cat_dict['name'] = cat['name']
        cat_dict['slug'] = cat['slug']
        cat_dict['items_count'] = cat_count
        cat_count_list.append(cat_dict)
        print(f"Total {cat['name']} {cat['slug']} Items {index+1}/{len(item_cat['children'])}: {cat_count}")

    except Exception as e:
        print(f"Failed {cat['name']} {cat['slug']} Items {index+1}/{len(item_cat['children'])}: {e}")
        cat_dict['status_code'] = response_cat_count.status_code
        cat_dict['id'] = cat['_id']
        cat_dict['name'] = cat['name']
        cat_dict['slug'] = cat['slug']
        cat_dict['items_count'] = 0
        cat_dict['error'] = e
        cat_count_list.append(cat_dict)

    time.sleep(0.5)

# Insert category_items to item_cat
item_cat['category_items'] = cat_count_list

# Dump to json
data_export_json(
    data=item_cat,
    content_date=content_date,
    website=website,
    folder_name=f'scraper/{website}',
    version=version,
    additional_info=f"scrape-cat_all",
    metadata={
        "status_code": response_cat.status_code,
        "total_items": total_items,
        "total_categories": len(item_cat['children']),
        "url_count": url_scrape_count,
        "url_categories": url_scrape_cat
    }
)

diff = abs(total_items - sum([i['items_count'] for i in item_cat['category_items']]))
if diff > 20:
    raise ValueError("Difference of Total Items vs Total Category Items more than 20:", diff)

### Scrape All Items on Skincare per Category

In [ ]:
# Loop all categories within Skincare
from math import ceil

for index, cat in enumerate(item_cat['category_items']):

    cat_items = cat['items_count']
    cat_name = cat['name']
    cat_id = cat['id']
    cat_slug = cat['slug']

    # Generate looping count for each category
    range_limit = 50 # default maximum limit per page
    page_total = ceil(cat_items/range_limit)

    print(f"Category {cat_name} {cat_slug}: {index+1}/{len(item_cat['category_items'])} | Category Items: {cat_items} | Range Limit: {range_limit} | Loop Page: {page_total}")

    for loop_count in range(page_total):
        page = loop_count+1
        headers = {
            'accept': 'application/json, text/plain, */*',
            'accept-language': 'en-US,en;q=0.9,id;q=0.8',
            'adjust_device_id': '0de186d9-af6f-44cc-27c5-d0b78ebaec19',
            'origin': 'https://www.sociolla.com',
            'priority': 'u=1, i',
            'referer': 'https://www.sociolla.com/',
            'sec-ch-ua': '"Chromium";v="142", "Google Chrome";v="142", "Not_A Brand";v="99"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'sec-fetch-dest': 'empty',
            'sec-fetch-mode': 'cors',
            'sec-fetch-site': 'same-site',
            'session_id': 'slwrj5hc-iul1-4nuu-bsty-5vnqflqtmcf5',
            'soc-platform': 'sociolla-web-mobile',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36',
            # 'cookie': 'sso_token=1b6df37b214caba6ac913e935d4e82f644ecfdb4; SOCIOLLA_UTM_DATA={}; SOCIOLLA_SESSION_ID=slwrj5hc-iul1-4nuu-bsty-5vnqflqtmcf5; _gid=GA1.2.1512964316.1764085677; _gcl_au=1.1.2118681921.1764085678; _fbp=fb.1.1764085684064.131487521762752714; _hjSession_1067100=eyJpZCI6IjM0ODRlODg1LTA0NTQtNGUyNi1hMTBlLTA3NmQ4NjhkNmEyZCIsImMiOjE3NjQwODU2ODQxNzMsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjoxLCJzcCI6MX0=; _tt_enable_cookie=1; _ttp=01KAXV6P6S7RFD10XVYF4EB3H4_.tt.1; _hjSessionUser_1067100=eyJpZCI6ImFhODUxY2M1LTUzZjktNTc2Yy05YWNmLWM5MmQyNmJjNjk4ZiIsImNyZWF0ZWQiOjE3NjQwODU2ODQxNzEsImV4aXN0aW5nIjp0cnVlfQ==; ttcsid=1764085684456::wNOTNM-WZilRf-bNQlPi.1.1764085724342.0; ttcsid_C10ST5FMJ1JP7306D860=1764085684455::bHsnIh5Apg41LRMq9u5J.1.1764085724343.0; ttcsid_C7FU0AI0VTGLO2V4FJK0=1764085684457::LhOq5jGx-4rzao-I8Mfe.1.1764085724343.0; _gat=1; _dc_gtm_UA-57294171-1=1; _ga_363PMMDMHM=GS2.1.s1764085678$o1$g1$t1764086157$j57$l0$h0; _ga=GA1.1.744826987.1764085677',
        }

        url_scrape = f'https://catalog-api.sociolla.com/search?limit={range_limit}&skip={range_limit*loop_count}&sort=-best-seller&filter=%7B%22categories.id%22:%22{cat_id}%22%7D'

        print(f"Get Category Items {cat_name} {cat_slug}: {index+1}/{len(item_cat['category_items'])} | Loop Page: {page}/{page_total} | URL: {url_scrape}")

        response = requests.get(
            url_scrape,
            headers=headers,
        )

        print(f"Response Category Items {cat_name} {cat_slug}: {index+1}/{len(item_cat['category_items'])} | Loop Page: {page}/{page_total} | URL: {url_scrape} | STATUS CODE: {response.status_code}")

        try:
            data_export_json(
                data=response.json(),
                website=website,
                folder_name=f'scraper/{website}',
                version=version,
                content_date=content_date, # "0000-00-00"
                additional_info=f"scrape-cat_{cat_id}-page{page}",
                metadata={
                    "status_code": response.status_code,
                    "cat_items": cat_items,
                    "cat_name": cat_name,
                    "cat_id": cat_id,
                    "cat_slug": cat_slug,
                    "page": page,
                    "url_scrape": url_scrape
                }
            )

        except Exception as e:
            data_export_json(
                data=response.text,
                website=website,
                folder_name=f'scraper/{website}',
                version=version,
                content_date=content_date, # "0000-00-00"
                additional_info=f"scrape-cat_{cat_id}-page{page}",
                metadata={
                    "status_code": response.status_code,
                    "error": e,
                    "cat_items": cat_items,
                    "cat_name": cat_name,
                    "cat_id": cat_id,
                    "cat_slug": cat_slug,
                    "page": page,
                    "url_scrape": url_scrape
                }
            )

        time.sleep(0.5)

    #     break
    # break

## Parser

In [ ]:
def parser_item_variant(
        response_json: dict,
        scrape_date: datetime,
        cat_name: str,
        cat_slug: str,
        page_cat: int,
        len_item_cat: int,
        page: int,
        page_total: int,
        url_scrape: str
    ):

    len_item = len(response_json['data'])
    parsed_list = list()
    for index, items in enumerate(response_json['data']):
        item_count = index+1

        # Item Dependencies Across Variants
        item_id = items['id']
        item_brand = items['brand']['name']
        item_name = items['name']
        try:
            item_url = items['url_sociolla']
        except KeyError as e:
            item_url = None
        item_slug = items['slug']
        # Item Ranking Dependencies
        review_total = items['review_stats']['total_reviews']
        review_rating = items['review_stats']['average_rating']
        review_recommended = items['review_stats']['total_recommended_count']
        try:
            wishlist = items['total_wishlist']
        except KeyError as e:
            wishlist = None
        price_range = items['price_range']

        len_var = len(items['combinations'])

        if len([v for v in items['combinations'] if 'attributes' in v.keys()]) > 0:
            is_package = False
            print(f"Parsing Category Items {cat_name} {cat_slug}: {page_cat}/{len_item_cat} | Loop Page: {page}/{page_total} | Loop Item: {item_count}/{len_item} | Count Variant: {len_var} | Is Package: {is_package} | URL: {url_scrape}")
            for index, c in enumerate(items['combinations']):
                if 'attributes' not in c.keys():
                    continue
                try:
                    # for attr in c['attributes'].keys():
                    try:
                        variant_id = [c['attributes'][attr]['my_soco_sql_id'] for attr in c['attributes'].keys()]
                    except KeyError:
                        variant_id = None
                    try:
                        variant_name = [c['attributes'][attr]['name'] for attr in c['attributes'].keys()]
                    except KeyError:
                        variant_name = None
                    if len(c['images']) > 0:
                        try:
                            url_image = [v['url'] for v in c['images'] if v['is_cover']][0]
                        except Exception:
                            try:
                                url_image = [v['url'] for v in c['images'] if not v['is_cover']][0]
                            except Exception:
                                url_image = [v['url'] for v in items['images']][0]
                    else:
                        try:
                            url_image = [v['url'] for v in items['images']][0]
                        except Exception:
                            url_image = None 

                    data_dict = dict()
                    data_dict['scrape_date'] = str(scrape_date)
                    data_dict['category_slug'] = cat_slug
                    data_dict['category'] = cat_name
                    data_dict['slug'] = item_slug
                    data_dict['id'] = item_id
                    data_dict['brand'] = item_brand
                    data_dict['name'] = item_name
                    data_dict['url'] = item_url
                    data_dict['url_image'] = url_image
                    data_dict['review_total'] = review_total
                    data_dict['review_rating'] = review_rating
                    data_dict['review_recommended'] = review_recommended
                    data_dict['wishlist'] = wishlist
                    data_dict['variant_id'] = variant_id
                    data_dict['variant_name'] = variant_name
                    data_dict['stock'] = c['stock']
                    data_dict['ean'] = c['ean_no']
                    data_dict['currency'] = 'IDR'
                    data_dict['price_range'] = price_range
                    data_dict['price'] = c['price']
                    data_dict['price_after_disc'] = c['price_after_discount']
                    data_dict['price_disc_type'] = c['deduction_type']
                    data_dict['price_disc'] = c['deduction_amount']
                    data_dict['price_disc_perc'] = c['deduction_percentage']
                    data_dict['is_package'] = is_package

                    parsed_list.append(data_dict)
                except Exception as e:
                    print(f"ERROR Parsing Category Items {cat_name} {cat_slug}: {page_cat}/{len_item_cat} | Loop Page: {page}/{page_total} | Loop Item: {item_count}/{len_item} | Count Variant: {len_var} | Is Package: {is_package} | Variant Index: {index} | {e}")
                    continue

        elif len(items['pack_detail']) > 0:
            is_package = True
            print(f"Parsing Category Items {cat_name} {cat_slug}: {page_cat}/{len_item_cat} | Loop Page: {page}/{page_total} | Loop Item: {item_count}/{len_item} | Count Variant: {len_var} | Is Package: {is_package} | URL: {url_scrape}")
            try:
                # for attr in items['default_combination']['attributes'].keys():
                try:
                    variant_id = [items['default_combination']['attributes'][attr]['my_soco_sql_id'] for attr in items['default_combination']['attributes'].keys()]
                except KeyError:
                    variant_id = None
                try:
                    variant_name = [items['default_combination']['attributes'][attr]['name'] for attr in items['default_combination']['attributes'].keys()]
                except KeyError:
                    variant_name = None
                if len(items['default_combination']['images']) > 0:
                    try:
                        url_image = [v['url'] for v in items['default_combination']['images'] if v['is_cover']][0]
                    except Exception:
                        try:
                            url_image = [v['url'] for v in items['default_combination']['images'] if not v['is_cover']][0]
                        except Exception:
                            url_image = [v['url'] for v in items['images']][0]
                else:
                    try:
                        url_image = [v['url'] for v in items['images']][0]
                    except Exception:
                        url_image = None 

                data_dict = dict()
                data_dict['scrape_date'] = str(scrape_date)
                data_dict['category_slug'] = cat_slug
                data_dict['category'] = cat_name
                data_dict['slug'] = item_slug
                data_dict['id'] = item_id
                data_dict['brand'] = item_brand
                data_dict['name'] = item_name
                data_dict['url'] = item_url
                data_dict['url_image'] = url_image
                data_dict['review_total'] = review_total
                data_dict['review_rating'] = review_rating
                data_dict['review_recommended'] = review_recommended
                data_dict['wishlist'] = wishlist
                data_dict['variant_id'] = variant_id
                data_dict['variant_name'] = variant_name
                data_dict['stock'] = items['default_combination']['stock']
                data_dict['ean'] = None
                data_dict['currency'] = 'IDR'
                data_dict['price_range'] = price_range
                data_dict['price'] = items['default_combination']['price']
                data_dict['price_after_disc'] = items['default_combination']['price_after_discount']
                data_dict['price_disc_type'] = items['default_combination']['deduction_type']
                data_dict['price_disc'] = items['default_combination']['deduction_amount']
                data_dict['price_disc_perc'] = items['default_combination']['deduction_percentage']
                data_dict['is_package'] = is_package

                parsed_list.append(data_dict)
            except Exception as e:
                print(f"ERROR Parsing Category Items {cat_name} {cat_slug}: {page_cat}/{len_item_cat} | Loop Page: {page}/{page_total} | Loop Item: {item_count}/{len_item} | Count Variant: {len_var} | Is Package: {is_package} | {e}")

        else:
            raise e

    return parsed_list

In [ ]:
# Get Item Cat Skincare
item_cat = data_import_json(
    content_date=content_date,
    website=website,
    folder_name=f'scraper/{website}',
    version=version,
    additional_info=f"scrape-cat_all"
)
item_cat = item_cat['data']

len_item_cat = len(item_cat['category_items'])

# Loop all categories within Skincare
from math import ceil

parsed_list = list()
try:
    for index, cat in enumerate(item_cat['category_items']):
        page_cat = index+1
        cat_items = cat['items_count']
        cat_name = cat['name']
        cat_id = cat['id']
        cat_slug = cat['slug']

        # Generate looping count for each category
        range_limit = 50 # default maximum limit per page
        page_total = ceil(cat_items/range_limit)

        print(f"Category {cat_name} {cat_slug}: {page_cat}/{len_item_cat} | Category Items: {cat_items} | Range Limit: {range_limit} | Page Total: {page_total}")

        for loop_count in range(page_total):
            page = loop_count+1
            url_scrape = f'https://catalog-api.sociolla.com/search?limit={range_limit}&skip={range_limit*loop_count}&sort=-best-seller&filter=%7B%22categories.id%22:%22{cat_id}%22%7D'

            try:
                response_json = data_import_json(
                    website=website,
                    folder_name=f'scraper/{website}',
                    version=version,
                    content_date=content_date, # "0000-00-00"
                    additional_info=f"scrape-cat_{cat_id}-page{page}"
                )
                response_json = response_json['data']

                # len_item = len(response_json['data'])
                # parsed_list = list()
                # for index, i in enumerate(response_json['data']):
                #     item_count = index+1

                parsed_list_temp = parser_item_variant(
                    response_json=response_json,
                    scrape_date=content_date,
                    cat_name=cat_name,
                    cat_slug=cat_slug,
                    page_cat=page_cat,
                    len_item_cat=len_item_cat,
                    page=page,
                    page_total=page_total,
                    url_scrape=url_scrape
                )

            except Exception as e:
                parsed_list_temp = list()
                print(f"ERROR Category Items {cat_name} {cat_slug}: {page_cat}/{len_item_cat} | Loop Page: {page}/{page_total} | URL: {url_scrape} | {e}")
                continue

            parsed_list = parsed_list + parsed_list_temp

    # Convert all list to DataFrame
    df_parse = pd.DataFrame(parsed_list)

    data_export_pandas(
        df_output=df_parse,
        website=website,
        folder_name=f'parser/{website}',
        version=version,
        content_date=content_date, # "0000-00-00"
        additional_info="parsed",
        incl_excel=True
    )

except Exception as e:
    print(e)

    df_parse = pd.DataFrame(parsed_list)
    data_export_pandas(
        df_output=df_parse,
        website=website,
        folder_name=f'parser/{website}',
        version=version,
        content_date=content_date, # "0000-00-00"
        additional_info="parsed",
        incl_excel=True
    )